In [116]:
#The code to transfer DAIL-SQL question organization to OneSQL organization.
import json
with open('./DAIL-SQL/dataset/process/BIRD-TEST_SQL_0-SHOT_CTX-200_ANS-2048_wEvidence/questions.json' , 'r') as f:
    generated_prompts_file_byte = f.read()
    generated_prompts = json.loads(generated_prompts_file_byte)

for idx, question in enumerate(generated_prompts['questions']):
    prompt = question['prompt']
    prompt = prompt.replace('/* Given the following database schema: */', '' )
    prompt = prompt.replace('/* Answer the following:' , '--')
    prompt = prompt.replace(' */' , '')
    if idx<5:
        print('Before: ' , question['prompt'])
        print('After : ' , prompt)
    generated_prompts['questions'][idx]['prompt'] = prompt


with open('./components/BIRD-TEST_SQL_0-SHOT_OneSQLStyle_wEvidence.json' , 'w' )as f:
    json.dump(generated_prompts , f)




In [72]:
#Converting OmniSQL style devset to codeS style devset.
with open('./Qwen_pred/SFTCodeS-7b_beam4_BIRD_wEvidence_0-5shot_CodeSORG.json' , 'r') as f:
    file_bytes = f.read()
    reference_json = json.loads(file_bytes)

with open('./OmniSQL/train_and_evaluate/data/dev_bird.json' , 'r') as f:
    file_bytes = f.read()
    prompts_json = json.loads(file_bytes)

for i in range(len(prompts_json)):
    prompts = prompts_json[i]['input_seq']
    reference_json['questions'][int(i)]['prompt'] = prompts

with open('./components/OmniSQL_BIRD_0-shot_wEvidence.json' , 'w' )as f:
    json.dump(reference_json , f)

In [107]:
#Code to make the dataset (DAIL style dataset) from CodeS predictions and the generated prompts
with open('./Qwen_pred/SFTCodeS-7b_beam4_BIRD_wEvidence_0-5shot_CodeSORG.json' , 'r') as f:
    file_bytes = f.read()
    reference_json = json.loads(file_bytes)

print(reference_json['questions'][0].keys())


with open('./codeS_pred/pred_codes-1b_BIRD_table_num_5_column_num_6_5-shot_max_tokens_8192_max_new_tokens_256.json' , 'r') as f:
    file_bytes = f.read()
    pred_json = json.loads(file_bytes)
i = 0

with open('./CodeS/codes/prompts_codes-1b_EVIDENCE_table_num_5_column_num_6_1-shot_max_tokens_8192_max_new_tokens_256.json' , 'r') as f:
    file_bytes = f.read()
    prompts_json = json.loads(file_bytes)

for i in pred_json.keys():
    response = pred_json[i]
    prompts = prompts_json[i]
    reference_json['questions'][int(i)]['prompt'] = prompts
    reference_json['questions'][int(i)]['response'] = response.split('----- bird -----')[0]

with open('./components/codes-1b_BIRD_table_num_5_column_num_6_1-shot_wEvidence_max_tokens_8192_max_new_tokens_256.json' , 'w' )as f:
    json.dump(reference_json , f)

dict_keys(['prompt_tokens', 'prompt', 'response', 'n_examples', 'db_id'])


In [420]:
#dividing the demonstrations into multiple dataset to make them our components for ensemble
#reading a reference bird file. We won't care about the response part but we only care about the prompt section and the answer id
import random
import copy

with open('./codeS_pred/codes-1b_BIRD_table_num_5_column_num_6_5-shot_max_tokens_8192_max_new_tokens_256.json' , 'r') as f:
    file_bytes = f.read()
    reference_json = json.loads(file_bytes)

ex_per_component = 5
components_name = [ 'codes-1b_BIRD_table_num_6_column_num_10_5-shot_0-5_wEvidence_max_tokens_8192_max_new_tokens_256.json',
                    'codes-1b_BIRD_table_num_6_column_num_10_5-shot_5-10_wEvidence_max_tokens_8192_max_new_tokens_256.json',
                    'codes-1b_BIRD_table_num_6_column_num_10_5-shot_10-15_wEvidence_max_tokens_8192_max_new_tokens_256.json',
                    'codes-1b_BIRD_table_num_6_column_num_10_5-shot_15-20_wEvidence_max_tokens_8192_max_new_tokens_256.json',
                    'codes-1b_BIRD_table_num_6_column_num_10_5-shot_20-25_wEvidence_max_tokens_8192_max_new_tokens_256.json'
                  ]
output_datasets = [ copy.deepcopy(reference_json) for i in range( len(components_name) ) ]

with open('./CodeS/codes/prompts_codes-1b_EVIDENCE_table_num_6_column_num_10_25-shot_max_tokens_8192_max_new_tokens_256.json' , 'r') as f:
    file_bytes = f.read()
    pred_json = json.loads(file_bytes)


for i in range( len(pred_json.keys()) ):
    prompt = pred_json[str(i)]
    splitting_term = 'database schema :'
    prompt_list = prompt.split( splitting_term )
    the_question = splitting_term + prompt_list[-1]
    examples = prompt_list[1:-1]
    last_notprocessd_ex = 0
    for component_index in range( len(components_name) ):
        prompt = ''
        for ex_index in range(ex_per_component):
            # chosen_example = random.choice(examples) # if you wanna select the examples randomly
            # examples.remove(chosen_example) # if you wanna select the examples randomly
            # prompt = prompt + splitting_term + chosen_example
            prompt = prompt + splitting_term + examples[ last_notprocessd_ex + ex_index ] #if the examples are supposed to be in order for each component
        prompt = prompt + the_question
        last_notprocessd_ex += ex_per_component
        output_datasets[component_index]['questions'][i]['prompt'] = prompt

for index , component_name in enumerate(components_name):
    with open( './components/' + component_name , 'w' )as f:
        json.dump(output_datasets[index] , f)

In [68]:
import pickle as pkl
with open('output_sequences-1.pkl', 'rb') as f:  # open a text file
    output_sequences = pkl.load(f)

In [77]:
import json
import time
def extract_prompt_from_list_of_questions(question_list):
    batch_list = []
    for i in question_list:
        batch_list.append(i['prompt'])
    return batch_list

from utils.post_process import get_exec_output

import psutil
def query_to_db(query , db_id , db_dir ):
    db_path = f"{db_dir}/{db_id}/{db_id}"
    flag, denotation = get_exec_output(
            db_path,
            query)
    
    return flag, denotation

from threading import Thread
import threading
import ctypes
class Thread_with_exception(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None , daemon=False):
        Thread.__init__(self, group, target, name, args, kwargs , daemon=daemon)
        
        self._return = None
            
    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
         
    def get_id(self):
        # returns id of the respective thread
        if hasattr(self, '_thread_id'):
            return self._thread_id
        for id, thread in threading._active.items():
            if thread is self:
                return id

    def join(self, *args):
        Thread.join(self, *args)
        return self._return
 
    def raise_exception(self):
        thread_id = self.get_id()
        res = ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id, ctypes.py_object(SystemExit))
        if res > 1:
            ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id, 0)
            print('Exception raise failure')
            

import time
from utils.post_process import result_eq

def is_queries_equal(testing_query , ground_truth_query , db_id , db_dir, timeout_time , gt_results = None):
    #Input:
        #time_out_time: integer: time in seconds
    
    if ground_truth_query!='':
        with open('./log.txt', 'a') as f:
            f.write(f"procceing the ground_truth_query:\n{ground_truth_query}\n")
        start_time = time.time()
        
        gt_flag, gt_denotation = query_to_db(ground_truth_query , db_id , db_dir)
    
        gt_process_time = time.time() - start_time
        with open('./log.txt', 'a') as f:
            f.write(f"processing time: {gt_process_time}\n\n")
        gt_results = (gt_flag, gt_denotation , gt_process_time)
        
    else:
        gt_flag, gt_denotation, gt_process_time = gt_results

    max_timeout_time = max( timeout_time , 2*gt_process_time )
    print('query max timeout: ' , max_timeout_time)

    test_thread = Thread_with_exception(target= query_to_db , args = (testing_query , db_id , db_dir ) , daemon=True )
    with open('./log.txt', 'a') as f:
        f.write(f"proccessing the testing_query:\n{testing_query}\n")
    start_time = time.time()
    test_thread.start()
    test_result = test_thread.join(max_timeout_time)
    if test_thread.is_alive():
        with open('./log.txt', 'a') as f:
            f.write(f"**********processing is terminated due to timeout. max_timeout_time: {max_timeout_time}\n")
        test_thread.raise_exception()
        test_thread.join()
        return False, gt_results
        
    test_flag, test_denotation = test_result
    with open('./log.txt', 'a') as f:
        f.write(f"Processing time for the testing query: {time.time() - start_time}\n\n")
    
    if gt_flag[0] != 'result':
        with open('./log.txt', 'a') as f:
            f.write(f"!!!!!!!!!!The following ground truth has an error:\n{ground_truth_query}\n\n")
        return False, gt_results
    elif test_flag[0] != 'result':
        return False, gt_results
    elif 'ORDER BY' in ground_truth_query or 'order by' in ground_truth_query:
        is_equal = result_eq(gt_flag[1] , test_flag[1] , order_matters=True)
    else:
        is_equal = result_eq(gt_flag[1] , test_flag[1] , order_matters=False)
    
    return is_equal, gt_results
    
import os
def put_responses_back_to_json_dataset(index , json_dataset , sequences, dataset_type='spider' , timeout_time=1):#dataset_type=spider/bird
    #Inputs:
        #sequences: List of [ { 'generated_text' : gen_text } ] or [gen_text1 , gen_text2 , ...]
        #json_dataset: json dataset that contains the ground truth in its 'response' part
            #If the elements in input sequences are like seq[0]['generated_text'] then json_dataset should contain the prompt in 'prompts' part
    gt_result_cache_file = './cache/' + dataset_type + '_results.pkl'
    gt_results_is_cached = False
    gt_results_list = []
    
    if os.path.isfile(gt_result_cache_file):
        gt_results_is_cached = True
        with open(gt_result_cache_file , 'rb' )as f:
            gt_results_list = pkl.load(f)

    db_dir = './DAIL-SQL/dataset/'+ dataset_type +'/database'
    execution_accuracy = 0
    
    for i in range( 0, len(sequences), 1 ):
        seq = sequences[i]
        # print(f"Number of processed sequences: {i}\t|\tNumber of correct queries: {execution_accuracy} \n")
        # if i%10==0:
        with open('./log.txt', 'a') as f:
            f.write(f"Number of processed sequences: {i}\t|\tNumber of correct queries: {execution_accuracy} \n")
            
        prompt_len = len ( json_dataset['questions'][index+i]['prompt'] )
        if isinstance(seq, list):
            gen_text = seq[0]['generated_text'][prompt_len:]
        else:
            # gen_text = seq[prompt_len:]
            gen_text = seq
            
        processed_gen_text = post_process_get_sql_from_gentext(gen_text)
        
        db_id = json_dataset['questions'][index+i]['db_id']

        # if dataset_type=='spider':

        #     if gt_results_is_cached:
        #         is_equal , gt_results = is_queries_equal(processed_gen_text , '' , db_id , db_dir, timeout_time , gt_results = gt_results_list[i])
        #     else:
        #         ground_truth = post_process_get_sql_from_gentext( json_dataset['questions'][index+i]['response'] )
        #         is_equal , gt_results = is_queries_equal(processed_gen_text , ground_truth , db_id , db_dir, timeout_time )
        #         gt_results_list.append(gt_results)
        #     execution_accuracy += is_equal
            
        json_dataset['questions'][index+i]['response'] = processed_gen_text
        # print('is_equal: ', is_equal)
        # print('--------------------------')
    if not gt_results_is_cached:
        with open(gt_result_cache_file , 'wb' )as f:
            pkl.dump( gt_results_list , f )
        
    return execution_accuracy

from utils.post_process import process_duplication
def post_process_get_sql_from_gentext(gen_text):
    # remove \n and extra spaces
    # print(gen_text)
    sql = " ".join(gen_text.replace("\n", " ").split())
    sql = process_duplication(sql)
    # python version should >= 3.8
    if sql.startswith("SELECT"):
        sql = sql
    elif sql.startswith(" "):
        sql = "SELECT" + sql
    else:
        sql = "SELECT " + sql
    return sql

data_size=1034
def eval_list_sql(sql_list , groundtruth_json_file_name , output_filename='' , dataset_type='spider' ):
    #This function gets a list of sql predictions and evalueates it and creates a dail style dataset if the output_file_name is given
    # groundtruth_json_file_name: Address of a dail style dataset containing the groundtruth in its response part.
        #If the sql_list elements are of shape seq[0]['generated_text'] groundtruth_json_file_name should have the prompts too
    with open(groundtruth_json_file_name , 'r') as f:
        generated_prompts_file_byte = f.read()
        generated_prompts = json.loads(generated_prompts_file_byte)
        exec_acc = put_responses_back_to_json_dataset( 0 , generated_prompts , sql_list , dataset_type=dataset_type )
    print('execution accuracy = ' , exec_acc/data_size)
    with open('./log.txt', 'a') as f:
        f.write(f"execution accuracy = {exec_acc/data_size} \n")
    if output_filename !='':
        with open( output_filename , 'w' )as f:
            json.dump(generated_prompts , f)

def dail_dataset_to_response_list( dail_style_dataset_name ):
    #This function gets a dail-style dataset and returns the all the responses in that dataset in a list.
    return_list = []
    with open( dail_style_dataset_name , 'r') as f:
        generated_response_file_byte = f.read()
        generated_response = json.loads(generated_response_file_byte)
    for i in generated_response['questions']:
        return_list.append( i['response'] )
    return return_list
        
    

In [124]:
#re-evaluating a dataset that is created before
import json
groundtruth_json_file_name = './DAIL-SQL/dataset/process/SPIDER-TEST_SQL_0-SHOT_CTX-200_ANS-2048/questions.json'

evaluating_dataset = './llama_pred/SPIDER_beam_4_lenpen0-MBRdbScore-TEST_SQL_3-SHOT-5Components_ranks0_4_8_12_16_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048_llama_7b_v2.json'

output_sequences = dail_dataset_to_response_list( evaluating_dataset )

eval_list_sql(output_sequences , groundtruth_json_file_name , output_filename='' , dataset_type='spider' )


execution accuracy =  0.42649903288201163


In [36]:
#evaluating some generated prompts recently generated. This is for SPIDER
import pickle as pkl

groundtruth_json_file_name = './DAIL-SQL/dataset/process/SPIDER-TEST_SQL_3-SHOT_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048_llama_7b/questions.json'

# with open('./output_sequences-5_lenpen1.pkl', 'rb') as f:  # open a text file
#     output_sequences = pkl.load(f)

eval_list_sql(output_sequences , groundtruth_json_file_name , 
              output_filename='./llama_pred/SPIDER_beam_4_lenpen0-TEST_SQL_seq_level_TreeBased_regBeamSearch_Qwen7b_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048_llama_7b.json' , dataset_type='spider' )


execution accuracy =  0.0


In [106]:
#evaluating some generated prompts recently generated. This is for BIRD
import pickle as pkl

groundtruth_json_file_name = './DAIL-SQL/dataset/process/BIRD-TEST_SQL_0-SHOT_CTX-200_ANS-2048/questions.json'

# with open('./output_sequences-5_lenpen1.pkl', 'rb') as f:  # open a text file
#     output_sequences = pkl.load(f)

eval_list_sql(output_sequences , groundtruth_json_file_name , 
              output_filename='./Qwen_pred/OmniSQL-7b_greedy_BIRD_wEvidence_0shot_OmniORG_chatTemplate.json' , dataset_type='bird' )


execution accuracy =  0.0


In [105]:
import pickle as pkl
file_name = './final_output_CodeS_preprocess_ensemble.pkl'
with open(file_name , 'rb') as f:
    output_sequences = pkl.load(f)

print(len(output_sequences))

1534


In [99]:
#grouping the pieces of output_sequences
import pickle as pkl
prefix = './outputs1/output_sequences_'
output_sequences = []
for i in range(0 , 1050 , 50):
    if i <1000:
        if i == 0: 
            file_name = prefix + '0_'  + str(i+50).lstrip('0') + '-4.pkl'
        else:
            file_name = prefix + str(i).lstrip('0') + '_' + str(i+50).lstrip('0') + '-4.pkl'
    else:
        file_name = prefix + str(i).lstrip('0') + '_end' + '-4.pkl'
    print(file_name)
    with open(file_name , 'rb') as f:
        part_of_output = pkl.load(f)
        print(len(part_of_output))
        # new_part_of_output = []
        # if len(part_of_output)==170:
        #     for x in range(0,len(part_of_output),5):
        #         new_part_of_output.append(x)
        #     with open('./new_'+file_name[2:], 'wb')as h:
        #         pkl.dump(new_part_of_output , h)
    output_sequences.extend(part_of_output)

print( len(output_sequences) )

./outputs1/output_sequences_0_50-4.pkl
1000
./outputs1/output_sequences_50_100-4.pkl
1000
./outputs1/output_sequences_100_150-4.pkl
1000
./outputs1/output_sequences_150_200-4.pkl
1000
./outputs1/output_sequences_200_250-4.pkl
1000
./outputs1/output_sequences_250_300-4.pkl
1000
./outputs1/output_sequences_300_350-4.pkl
1000
./outputs1/output_sequences_350_400-4.pkl
1000
./outputs1/output_sequences_400_450-4.pkl
1000
./outputs1/output_sequences_450_500-4.pkl
1000
./outputs1/output_sequences_500_550-4.pkl
1000
./outputs1/output_sequences_550_600-4.pkl
1000
./outputs1/output_sequences_600_650-4.pkl
1000
./outputs1/output_sequences_650_700-4.pkl
1000
./outputs1/output_sequences_700_750-4.pkl
1000
./outputs1/output_sequences_750_800-4.pkl
1000
./outputs1/output_sequences_800_850-4.pkl
1000
./outputs1/output_sequences_850_900-4.pkl
1000
./outputs1/output_sequences_900_950-4.pkl
1000
./outputs1/output_sequences_950_1000-4.pkl
1000
./outputs1/output_sequences_1000_end-4.pkl
680
20680


In [544]:
import pickle as pkl
with open('./vulcan_output/output_CodeS_beam4_bird_wEvidence_2shot_SFTCodeS-7b_2ndOf0-5shot_EOSbefore_batch_text.pkl', 'rb') as f:
    batch_text = pkl.load(f)
x = 5
print(batch_text[x])

Congrats! You have successfully answered the following questions. You are very knowledgeable about Databases and SQL.
<|endoftext|>database schema :
table staff , columns = [ staff.email ( text | comment : email of the staff ) , staff.first_name ( text | values : Mike , Jon ) , staff.password ( text ) , staff.address_id ( integer | values : 3 , 4 ) , staff.last_update ( datetime | values : 2006-02-15 04:57:16.0 ) , staff.store_id ( integer | values : 1 , 2 ) , staff.picture ( blob | comment : picture of the staff ) , staff.username ( text | values : Mike , Jon ) , staff.staff_id ( integer | primary key | values : 1 , 2 ) , staff.active ( integer | values : 1 ) ]
table film_actor , columns = [ film_actor.actor_id ( integer | primary key | values : 1 , 2 ) , film_actor.film_id ( integer | primary key | values : 1 , 23 ) , film_actor.last_update ( datetime | values : 2006-02-15 05:05:03.0 ) ]
table language , columns = [ language.language_id ( integer | primary key | values : 1 , 2 ) , la

In [408]:
with open('./vulcan_output/output_CodeS_beam4_bird_wEvidence_0shotFrom0shot_SFTCodeS-7b_padTrue_batch_text.pkl', 'rb') as f:
    batch_text = pkl.load(f)

print(batch_text[x])

<|endoftext|>database schema : table products , columns = [ products.productid ( integer | primary key | values : 1 , 2 ) , products.description ( text | values : Rucní zadání , Nafta ) ]
table yearmonth , columns = [ yearmonth.consumption ( real | values : 528.3 , 1598.28 ) , yearmonth.date ( text | primary key | values : 201112 , 201201 ) , yearmonth.customerid ( integer | primary key | values : 39 , 63 ) ]
table transactions_1k , columns = [ transactions_1k.productid ( integer | values : 2 , 23 ) , transactions_1k.date ( date | values : 2012-08-24 , 2012-08-23 ) , transactions_1k.customerid ( integer | values : 31543 , 46707 ) , transactions_1k.transactionid ( integer | primary key | values : 1 , 2 ) , transactions_1k.time ( text | values : 09:41:00 , 10:03:00 ) , transactions_1k.cardid ( integer | values : 486621 , 550134 ) , transactions_1k.price ( real | values : 672.64 , 430.72 ) , transactions_1k.amount ( integer | values : 28 , 18 ) , transactions_1k.gasstationid ( integer | v

In [95]:
with open('MBRoutput_sequences_total-4.pkl' , 'rb') as f:
    seqs = pkl.load(f)

total_distinced_candid = 0
for i in seqs:
    my_dict = dict()
    for t in i:
        for c in t:
            my_dict[c] = 1
    num_dist_candid = len(my_dict.keys())
    total_distinced_candid+=num_dist_candid

print(f'avg number of distinced candidates: {total_distinced_candid/len(seqs)}')
# print(len(seqs))

avg number of distinced candidates: 9.132333767926989
